我们知道地球系统明显的收到太阳(年)和月球(潮汐)的印象， 直观体现为就是时间可以用「太阳直射角」（太阳在天空中的高度）来描述，这样的话我们就可以按照 mesh 的方案构建一个 （32x64） 或者 （720x1440） 的时间mapping 而不是一个数字

# Solar 

我们首先要知道的是任意时间 (xxxx年xxxx日xxxx小时) 太阳直射点的经纬度。
- 太陽直射點是太陽高度角（太陽光入射角度）為90度的點，它是地心與日心連線和地球球面成90度的交點。
太陽直射點所在的經線的地方時理論上為正午12時，但需視乎該地政府所定下的時區為多少而定。

所以维度由公转决定， 精度由自转决定，我们先来算纬度


太阳赤纬=0.3723＋23.2567sinθ＋0.1149sin2θ－0.1712sin3θ－0.758cosθ＋0.3656cos
2θ＋0.0201cos3θ
式中θ称日角，即 θ=2πt／365.2422
这里t又由两部分组成，即 t=N－N0
式中N为积日，所谓积日，就是日期在年内的顺序号，例如，1月1日其积日为1，平年12月
31日的积日为365，闰年则为366，等等。
N0=79.6764＋0.2422×（年份－1985）－INT〔（年份－1985）／4〕

cite：http://m.blog.chinaunix.net/uid-8015611-id-2035001.html

In [5]:
import math

In [25]:
def jiri(year,month,day):
    if year%4==0 and year%100!=0 or year%400==0:leap=1
    else:leap=0
    mm=(0,31,leap+59,leap+90,leap+120,leap+151,leap+181,leap+212,leap+243,leap+273,leap+304,leap+334)
    return mm[month-1]+day

def solarpoint(year,month,day):
    t=jiri(year,month,day)-79.6764-0.2422*(year-1985)+int((year-1985)/4)
    sita=2*3.14159265*t/365.2422
    delta=0.3723+23.2567*math.sin(sita)+0.1149*math.sin(2*sita)-0.1712*math.sin(3*sita)-0.758*math.cos(sita)+0.3656*math.cos(
2*sita)+0.0201*math.cos(3*sita)
    return delta


In [19]:
import numpy as np

所以在 `np.datetime64("2018-01-01T12",'h')` 这个时间点的太阳直射点就是 w=`solarpoint(2018,1,1)` 经度就是 h=`(12.0 - currTime)*15.0` = 0

对于每一个太阳经纬度$(\theta,\phi)$, 我们都可以做出一个从「直射点」到「直射点背面」的一个演化mask。 值从 1 到 -1。 最简单的做法就是找向量的夹角。

In [ ]:
以 32x64 为例， 我们的数据是从 721x1440 的数据中跳出来的，所以具体的经纬度就是 np.

In [53]:
sH = 720
sW = 1440
tH = 32
tW = 64
steph = sH /float(tH)
stepw = sW /float(tW)
x     = np.arange(0, sW, stepw).astype('int')
y     = np.linspace(0, sH-1, tH, dtype='int')
x, y  = np.meshgrid(x, y)

latitude   = np.linspace(90,-90,721)
longitude  = np.linspace(0,360,1440)

LaLotude = np.stack([latitude[y],longitude[x]])/180*np.pi

LaLotudeVector = np.stack([np.cos(LaLotude[1])*np.cos(LaLotude[0]),np.cos(LaLotude[1])*np.sin(LaLotude[0]),np.sin(LaLotude[1])],2)

In [66]:
LaLotudeVector[0,0]

array([6.123234e-17, 1.000000e+00, 0.000000e+00])

In [75]:
phi = solarpoint(2018,1,1)/180*np.pi
theta   = (12.0 - 0)*15.0/180*np.pi

In [78]:
theta

3.141592653589793

In [76]:
solarpointvector=  np.stack([np.cos(theta)*np.cos(phi),np.cos(theta)*np.sin(phi),np.sin(theta)])

# luna system

月面系统会比较复杂一点， 有两个比较直观可用的工具
- https://astronomy.stackexchange.com/questions/29932/how-to-calculate-declination-of-moon
- https://astronomy.stackexchange.com/questions/30355/how-to-calculate-the-ground-track-of-the-moons-position-on-the-earths-surface
- 或者 Mathmatica 中的 `MoonPosition` 
- https://skymarvels.com/infopages/vids/Earth%20-%20Sub-lunar%20Point%20001.htm Demo

Given a date and time, the position of the Moon can be calculated to provide the declination and right ascension. The sub-point of the Moon (the point on the Earth at which the Moon is at the zenith) is as follows:

- latitude = declination of the Moon

- longitude can be found by calculating the local mean sidereal time (LMST) that equals the Moon's right ascension. (LMST dependents on the date, time, and longitude.) Everything is known except for the longitude.

简单来说就是有
$$
u = t - 0.48 \sin {2 \pi (t - 3.45) \over 27.55455}
$$

#### longitude

基本上就是用公式
$$
RA_{Moon}=LMST=GMST+long_{East}
$$

以及一个经验公式
$$
\mathit{GMST}_{\text{deg}} = 100.4606184 + 0.9856473662862 D + 15 H
$$

In [ ]:
这里的 D 是距离 Jan 1, 2000 at 12 h UT 的天数

Right ascension is generally measured in units of time, not degrees of arc. 24 hours of time corresponds to 360 degrees of arc at the equator. 

In [ ]:
基本上 RA 就是用 Greenwich Time 算出来的角度.


In [83]:
!pip install ephem

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.5 MB/s eta 0:00:00m eta 0:00:010:01:01


In [86]:
import ephem

In [88]:
import datetime

In [94]:
def get_sub_luna_point(year,month,day):
    greenwich = ephem.Observer()
    greenwich.lat = "0"
    greenwich.lon = "0"
    greenwich.date =datetime.datetime(year, month, day)

    #add Moon Sub Solar Point
    moon = ephem.Moon(greenwich)
    moon.compute(greenwich.date)
    moon_lon = math.degrees(moon.ra - greenwich.sidereal_time() )
    # map longitude value from -180 to +180 
    if moon_lon < -180.0 :moon_lon = 360.0 + moon_lon 
    elif moon_lon > 180.0 :moon_lon = moon_lon - 360.0

    moon_lat = math.degrees(moon.dec)
    #print( "moon Lon:",moon_lon, "Lat:",moon_lat)
    return moon_lon,moon_lat
    

In [98]:
def get_sub_sun_point(year,month,day):
    greenwich      = ephem.Observer()
    greenwich.lat  = "0"
    greenwich.lon  = "0"
    greenwich.date =datetime.datetime(year, month, day)

    #add sun Sub Solar Point
    sun = ephem.Sun(greenwich)
    sun.compute(greenwich.date)
    sun_lon = math.degrees(sun.ra - greenwich.sidereal_time() )
    # map longitude value from -180 to +180 
    if sun_lon < -180.0 :sun_lon = 360.0 + sun_lon 
    elif sun_lon > 180.0 :sun_lon = sun_lon - 360.0

    sun_lat = math.degrees(sun.dec)
    #print( "sun Lon:",sun_lon, "Lat:",sun_lat)
    return sun_lon,sun_lat